<h1><b>This Jupiter Notebook will be used to contain the Neighbourhood Selection project for the Coursera Data Science Capstone Project</b></h1>

In [1]:
import pandas as pd #Import Pandas with pd as alias.

In [2]:
import numpy as np #Import Numpy with np as alias.

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h3><i>Step 1: Webscrape Toronto location data from Postal list site</i></h3>

In [5]:
# Get additional libraries
!pip install bs4
!pip install requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#Pop in all the modules we could use.

from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

# code from Assignment
#!conda install -c conda-forge geocoder --yes  
!pip install geocoder
import geocoder # import geocoder

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

#import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

toml-0.10.2          | 18 KB     | ##################################### | 100% 
openjpeg-2.4.0       | 444 KB    | ##################################### | 100% 
pooch-1.3.0          | 40 KB     | ##################################### | 100% 
mkl_random-1.2.0     | 313 KB    | ##################################### | 100% 
zeromq-4.3.4         | 352 KB    | ##################################### | 100% 
psycopg2-2.8.6       | 169 KB    | ##################################### | 100% 
google-auth-1.30.0   | 77 KB     | ##################################### | 100% 
rsa-4.7.2            | 28 KB     | ##################################### | 100% 
ipython-7.23.1       | 1.1 MB    | ##################################### | 100% 
lz4-c-1.9.3          | 179 KB    | ##################################### | 100% 
tornado-6.1          | 646 KB    | ##################################### | 100% 
libvorbis-1.3.7      | 280 KB    | ##################################### | 100% 
lcms2-2.12           | 443 K

mysql-libs-8.0.23    | 1.8 MB    | ##################################### | 100% 
webencodings-0.5.1   | 12 KB     | ##################################### | 100% 
libzopfli-1.0.3      | 164 KB    | ##################################### | 100% 
libnghttp2-1.43.0    | 808 KB    | ##################################### | 100% 
nest-asyncio-1.5.1   | 9 KB      | ##################################### | 100% 
aiohttp-3.7.4        | 632 KB    | ##################################### | 100% 
jupyter_client-6.1.1 | 79 KB     | ##################################### | 100% 
libogg-1.3.4         | 206 KB    | ##################################### | 100% 
tk-8.6.10            | 3.2 MB    | ##################################### | 100% 
kiwisolver-1.3.1     | 78 KB     | ##################################### | 100% 
pyqt5-sip-4.19.18    | 310 KB    | ##################################### | 100% 
sip-4.19.25          | 295 KB    | ##################################### | 100% 
ca-certificates-2020 | 137 K

In [6]:
# Screen Grab. Add raw data column to help descramble borough/Neighborhood issues M3B,M3C,M3K,M3L.M3M,M3N,
Torontodf = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood','Raw'])
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
#find a html table in the web page
table = soup.find('table') # in html table is represented by the tag <table>
for row in table.findAll('td'):
    cell = {}    
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Raw'] = repr(row.span)
        cell['Borough'] = (row.span.text).split('(')[0]
#        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cell['Neighborhood'] = row.span.text.split('(')[1].replace(')',' ').replace(' /',',').strip(')')
        Torontodf = Torontodf.append({"PostalCode":cell['PostalCode'], "Borough":cell['Borough'], "Neighborhood":cell['Neighborhood'], "Raw":cell['Raw']}, ignore_index=True)
        if row.p.text[:3] == 'M3C':
            print(row.span)


<span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Don_Mills" title="Don Mills">Don Mills</a>)<br/>South<br/>(<a href="/wiki/Flemingdon_Park" title="Flemingdon Park">Flemingdon Park</a>)</span>


Clean Webscraped data.

In [7]:
# List of Borough replacements
Torontodf['Borough']=Torontodf['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
Torontodf.loc[Torontodf.PostalCode == 'M3C', 'Neighborhood'] = "Don Mills South, Flemingdon Park" # 2nd hood dropped in scrape
Torontodf.loc[Torontodf.PostalCode == 'M4J', 'Borough'] = "East York, East Toronto" # Postal code has 2 Boroughs can't drop / character.
Torontodf.loc[Torontodf.PostalCode == 'M3K', 'Neighborhood'] = "Downsview East, CFB Toronto" # 2nd hood dropped in scrape
Torontodf.loc[Torontodf.PostalCode == 'M5P', 'Neighborhood'] = "Forest Hill North, Forest Hill West" # Guess that Forest Hill North & West is FHN and FHW..
Torontodf.loc[Torontodf.PostalCode == 'M7R', 'Neighborhood'] = "Canada Post Gateway Processing Centre" # Cleaned up name
Torontodf.loc[Torontodf.PostalCode == 'M5W', 'Neighborhood'] = "Canada Post PO Boxes" # Cleaned up name
Torontodf.loc[Torontodf.PostalCode == 'M7Y', 'Neighborhood'] = "Canada Post Business Processing Centre" # Cleaned up name
Torontodf.loc[Torontodf.PostalCode == 'M7Y', 'Borough'] = "East Toronto" # Cleaned up name

#Drop Canada Post records because they aren't proper neighborhoods.
Torontodf.drop(Torontodf[Torontodf['PostalCode'] == 'M7R'].index, inplace = True)
Torontodf.drop(Torontodf[Torontodf['PostalCode'] == 'M5W'].index, inplace = True)
Torontodf.drop(Torontodf[Torontodf['PostalCode'] == 'M7Y'].index, inplace = True)

Torontodf.drop(columns='Raw', inplace=True)

#Torontodf.loc[Torontodf['PostalCode'].str.endswith('Z')]

#print(Torontodf['Neighborhood'])
#print(Torontodf.loc[Torontodf['PostalCode'].isin(['M3B','M3C','M3K','M4J','M3L','M3M','M3N'])])
#Torontodf.loc[Torontodf['PostalCode'].isin(['M3B','M3C','M3K','M4J','M3L','M3M','M3N'])]

In [8]:
Torontodf.shape

(100, 3)

Get Lat and Long data Added to DataFrame

NOTE: Geocoder is now part of arcgis, not google. Google link doesn't work.

In [9]:
#test geocoder 
g = geocoder.arcgis('{}, Toronto, Ontario'.format('M3C'))
print(g.latlng[0],' ',g.latlng[1])

43.72168000000005   -79.34351999999996


In [59]:
# Take the stock geocoder stats, and make a function out of them.
def Toronto_Lat_Lng(postal_code):
    # initialize your variable to None
 #   lat_lng_coords = None
 #   count = 0

    # loop until you get the coordinates
 #   while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
#        if g.latlng:
    return dict(Latitude = [g.latlng[0]], Longitude = [g.latlng[1]], code = [postal_code])
#        lat_lng_coords = g.latlng
#        count = count + 1
#    res = dict(Latitude = lat_lng_coords[0], Longitude = lat_lng_coords[1], num = count, code = postal_code)
#    return res

In [65]:
test = Toronto_Lat_Lng('M7R') # use M7R as a seed value to get the column types defined properly. (will remove later.)
print(test)
latlong = pd.DataFrame(test, index=[0])
#latlong = pd.DataFrame.from_dict(test, orient='index').T
#Latlong = pd.DataFrame(test, index=None)
latlong
#print('Lat is {0} and long is {1} and it took {2} tries.'.format(test[lat],test[long],test[num]))


{'Latitude': [43.648690000000045], 'Longitude': [-79.38543999999996], 'code': ['M7R']}


AttributeError: 'NoneType' object has no attribute 'items'

   Latitude  Longitude code
0  43.64869  -79.38544  M7R

EXCELLENT!! The function works. (Note: found in discussion forms something about geocoder.google is dead, and to use geocoder.arcgis. Fixed timeout error.

To get the below code working, we now get another attribute error, but it does work, so no harm no foul.

In [66]:
#latlong = pd.DataFrame.from_dict(Toronto_Lat_Lng('M7R'))

for i, row in Torontodf.iterrows():
  #  val = Toronto_Lat_Lng(row['PostalCode'])
    latlong = latlong.append([Toronto_Lat_Lng(row['PostalCode'])], ignore_index=True)
#print(row['PostalCode'])
#print('Total calls: {0}'.format(count))    
latlong

AttributeError: 'NoneType' object has no attribute 'items'

                 Latitude             Longitude   code
0                 43.6487              -79.3854    M7R
1     [43.75245000000007]  [-79.32990999999998]  [M3A]
2     [43.73057000000006]  [-79.31305999999995]  [M4A]
3     [43.65512000000007]  [-79.36263999999994]  [M5A]
4     [43.72327000000007]  [-79.45041999999995]  [M6A]
5     [43.66253000000006]  [-79.39187999999996]  [M7A]
6    [43.662630000000036]  [-79.52830999999998]  [M9A]
7    [43.811390000000074]  [-79.19661999999994]  [M1B]
8     [43.74923000000007]  [-79.36185999999998]  [M3B]
9     [43.70718000000005]  [-79.31191999999999]  [M4B]
10    [43.65739000000008]  [-79.37803999999994]  [M5B]
11    [43.70687000000004]  [-79.44811999999996]  [M6B]
12    [43.65034000000003]  [-79.55361999999997]  [M9B]
13    [43.78574000000003]  [-79.15874999999994]  [M1C]
14    [43.72168000000005]  [-79.34351999999996]  [M3C]
15    [43.68970000000007]  [-79.30681999999996]  [M4C]
16    [43.65215000000006]  [-79.37586999999996]  [M5C]
17    [43.

In [33]:
#Now we have our lats and longs.
latlong.drop(latlong[latlong['code'] == 'M7R'].index, inplace = True) # to remove the M7R we added temporarily
#latlong.rename(columns = {'code':'PostalCode'}, inplace = True) #rename the code column
#latlong.drop(columns = 'num', inplace = True)
finaldf = Torontodf.merge(latlong, left_on='PostalCode', right_on='code')
finaldf.drop(columns = 'code', inplace = True) # and get rid of that code now that it's served it's purpose in the merge.
finaldf

AttributeError: 'NoneType' object has no attribute 'items'

   PostalCode                  Borough  \
0         M3A               North York   
1         M4A               North York   
2         M5A         Downtown Toronto   
3         M6A               North York   
4         M7A             Queen's Park   
5         M9A                Etobicoke   
6         M1B              Scarborough   
7         M3B               North York   
8         M4B                East York   
9         M5B         Downtown Toronto   
10        M6B               North York   
11        M9B                Etobicoke   
12        M1C              Scarborough   
13        M3C               North York   
14        M4C                East York   
15        M5C         Downtown Toronto   
16        M6C                     York   
17        M9C                Etobicoke   
18        M1E              Scarborough   
19        M4E             East Toronto   
20        M5E         Downtown Toronto   
21        M6E                     York   
22        M1G              Scarbor

<b><h3>The set of Geospatial data is done. </h3></b>

Onto Viewing the Neighborhood/Borough data in Folium

In [68]:
# Get Toronto Lat and Long
latitude = 43.72168000000005
longitude = -79.34351999999996
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(finaldf['Latitude'], finaldf['Longitude'], finaldf['Borough'], finaldf['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [107]:
# test to display map through git hub. There is a trust issue between github and Jupiter
map_Toronto.save("Toronto.html")

Begin process of clustering (k-means) postal codes into regions. 

In [103]:
# first delete Cluster label columns as needed..
finaldf = finaldf.drop('Cluster Labels',1)
Toronto = Toronto.drop('Cluster Labels',1)
#print(finaldf)
#print(Toronto)

In [104]:
#Pull out Lat/long for each PostalCode set number of clusters
print(finaldf['Borough'].unique())
kclusters = 11  #Simply choose the number of boroughs, not counting the extra east york/east toronto borough

Toronto_grouped_clustering = finaldf.drop(['Neighborhood','Borough','PostalCode'], 1)

# run k-means clustering
kmeans = KMeans(init="k-means++", n_clusters=kclusters, random_state=0)

kmeans.fit(Toronto_grouped_clustering)

# add clustering labels  
Toronto = finaldf
Toronto.insert(0, 'Cluster Labels', kmeans.labels_)

k_means_cluster_centers = kmeans.cluster_centers_
k_means_cluster_centers

['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto'
 'East York, East Toronto' 'Central Toronto' 'Etobicoke Northwest']


array([[ 43.79925   , -79.1866875 ],
       [ 43.67140444, -79.45503444],
       [ 43.78017286, -79.29517286],
       [ 43.70307462, -79.38283154],
       [ 43.63546875, -79.52805625],
       [ 43.73947125, -79.45724375],
       [ 43.724533  , -79.540337  ],
       [ 43.69230444, -79.30675667],
       [ 43.77223625, -79.3809725 ],
       [ 43.746288  , -79.237454  ],
       [ 43.65366368, -79.38768053]])

In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters 
# Color selection leans towards blue, so the slight variation between the two light blue regions in the NW is a bit unclear.
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [106]:
# Hopefully this will get my map to display, using nbviewer
map_clusters.save("Toronto_clusters.html")

In [ ]:
# URL for nbViewer.. looks like that is the only way to get the maps to display.
# https://nbviewer.jupyter.org/github/Caimbul/Coursera_Capstone/blob/master/Neighbourhood%20Selection%20Jupiter%20Notebook.ipynb